# Scraping journalists then using twint to get who they follow

In [1]:
import sys
import os
import time
import json
import csv
import threading
import queue
import asyncio
import nest_asyncio
nest_asyncio.apply()
import twint

sys.path.insert(1, '../')
from src.data import journalists as journos

### 1. Getting journalist twitter handles according to topic

In [2]:
keyword = 'politic'
journo_handles = journos.get_handles_by_keyword(keyword)
print(len(journo_handles))

449


In [3]:
print(journo_handles)

['ShippersUnbound', 'TomMcTague', 'paulcockerton', 'NicolaRBartlett', 'DelMody', 'patrickwintour', 'dansabbagh', 'james_randerson', 'peterwalker99', 'hugh_muir', 'JohnDomokos', 'NigelpMorris', 'CharlieCooper8', 'Davidmkeys', 'tobyhelm', 'DavidWooding', 'elashton', 'ryansabey', 'katyballs', 'edwardmalnick', 'SheridanDani', 'elliotttimes', 'patrick_kidd', 'SamCoatesSky', 'PickardJE', 'khalafroula', 'robertshrimsley', 'RKWinvisibleman', 'Steven_Swinford', 'TimRoss_1', 'simon_telegraph', 'gordonrayner', 'nadiakhomami', 'JBeattieMirror', 'NigelNelson', 'RJPartington', 'GuardianHeather', 'FinanceJames', 'trevadavies', 'benglaze', 'mudiek', 'christopherhope', 'stephenkb', 'ladyhaja', 'tnewtondunn', 'ChrisMusson', 'LOS_Fisher', 'jennirsl', 'martinbeckford', 'johnestevens', 'danbloom1', 'philipstephens', 'darrendodd', 'tompeck', 'TMScotExpress', 'andrewrawnsley', 'nicholaswatt', 'martinkettle', 'fperraudin', 'mikeysmith', 'tamcohen', 'bbcnickrobinson', 'afneil', 'bbclaurak', 'rosschawkins', 'ch

## 2. Loop over journalists and get who they follow

### 2.2 Storing each threads results as csv files (USE THIS)

This seems to work quite nicely, and may even be more reliable than storing in memory.  

Note that any recursive use to get data that was missed has to be done manually by alternately using the last two cells before section 3 to recheck the failed list and run it.

In [4]:
num_threads = 6 # num_threads can be any amount, but only so many can be run in parallel, according to number of cores you have.
# import multiprocessing; multiprocessing.cpu_count() can tell you how many cores python can see.

In [5]:
def get_friends(q, fp, persist):
    '''
    Params
    ------
    q : a Queue instance
    
    fp : string
        Provides the path where the file should be saved.
    persist : bool
        If True, make repeat attempts until twint runs successfully.
    '''
    while True: 
        username = q.get()
        success = False
        filepath = fp+'friends_'+username+'.csv'
        while not success:
            print('Attempting to get friends of @'+username)
            c = twint.Config()
            c.Username = username
            c.User_full = False
            c.Hide_output = True
            c.Output = filepath

            twint.run.Following(c)
            
            if persist:
                if os.path.exists(filepath):
                    success = True
                    print('Friends of @'+username+' saved.')
            else:
                success = True
        q.task_done()

In [ ]:
q = queue.Queue(maxsize=0)

for i in range(num_threads): # Loop to create threads
    worker = threading.Thread(target=get_friends, args=(q, '../data/raw/'+keyword+'_', True)) # Change persist to True to tell the code to keep trying until results are written to file.
    worker.setDaemon(True)
    worker.start()

for username in journo_handles: # Loop to add journalists usernames to the queue
    q.put(username)

q.join()

Attempting to get friends of @ShippersUnboundAttempting to get friends of @TomMcTagueAttempting to get friends of @paulcockerton
Attempting to get friends of @NicolaRBartlett
Attempting to get friends of @DelMody
Attempting to get friends of @patrickwintour

Attempting to get friends of @dansabbagh

Attempting to get friends of @james_randerson


CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.feed:Follow:IndexError


Attempting to get friends of @TomMcTague


CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.feed:Follow:IndexError


Attempting to get friends of @NicolaRBartlett


...

Attempting to get friends of @P_Madeley_Star


CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.feed:Follow:IndexError


Attempting to get friends of @P_Madeley_Star


CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.feed:Follow:IndexError


Attempting to get friends of @P_Madeley_Star


In [11]:
all_handles = []
all_users = []
failed = []
for name in journo_handles:
    filepath = '../data/raw/'+keyword+'_friends_'+name+'.csv'
    if not os.path.exists(filepath):
        failed.append(name)
    else:
        with open(filepath, newline='') as f:
            reader = csv.reader(f)
            handles = list(reader)
            all_handles.extend([handle[0] for handle in handles])
            all_users.extend([name for handle in handles])
            print('@'+name+' follows '+str(len(handles))+' users.')
            
print('\nTotal number of handles pulled: '+str(len(all_handles)))

unique = len(set(all_handles))
print('Number of unique twitter handles: '+str(unique))

print('\nZero following in list for users: '+str(failed))

@ShippersUnbound follows 3735 users.
@TomMcTague follows 3947 users.
@paulcockerton follows 2459 users.
@NicolaRBartlett follows 5327 users.
@DelMody follows 2973 users.
@patrickwintour follows 1036 users.
@dansabbagh follows 2980 users.
@james_randerson follows 2420 users.
@peterwalker99 follows 729 users.
@hugh_muir follows 463 users.
@JohnDomokos follows 565 users.
@NigelpMorris follows 1027 users.
@CharlieCooper8 follows 1627 users.
@Davidmkeys follows 166 users.
@tobyhelm follows 2092 users.
@DavidWooding follows 1297 users.
@elashton follows 1783 users.
@ryansabey follows 4507 users.
@katyballs follows 2070 users.
@edwardmalnick follows 994 users.
@SheridanDani follows 1218 users.
@elliotttimes follows 1244 users.
@patrick_kidd follows 2638 users.
@SamCoatesSky follows 4983 users.
@PickardJE follows 4487 users.
@khalafroula follows 1838 users.
@robertshrimsley follows 1068 users.
@RKWinvisibleman follows 5051 users.
@Steven_Swinford follows 810 users.
@TimRoss_1 follows 1527 user

In [7]:
del all_handles
del all_users

In [9]:
failed.remove('P_Madeley_Star') # Is a protected account
failed.remove('AndyRoocroft')

If some of the requests seem to have failed, can try them again:

In [10]:
q = queue.Queue(maxsize=0)

for i in range(num_threads): # Loop to create threads
    worker = threading.Thread(target=get_friends, args=(q, '../data/raw/'+keyword+'_', True))
    worker.setDaemon(True)
    worker.start()

for username in failed: # Loop to add journalists usernames to the queue
    q.put(username)

q.join()

Attempting to get friends of @HannahWilkinso9Attempting to get friends of @GregoryHoare
Attempting to get friends of @KenReid_utv
Attempting to get friends of @Tracey_utv
Attempting to get friends of @BROPHONTHEBEAT

Attempting to get friends of @BBCHelenCatt


CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.feed:Follow:IndexError


Attempting to get friends of @BBCHelenCatt


CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.feed:Follow:IndexError


Attempting to get friends of @HannahWilkinso9


...

Friends of @Jimmief95 saved.


CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.feed:Follow:IndexError


Friends of @WritesBright saved.


CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.feed:Follow:IndexError


Friends of @andrewiconnell saved.


Interestingly, most of the handles must appear only once, suggesting there are only a few users that are followed by more than one of the journalists in this list.

## 3. Store the data in one csv file using pandas

In [15]:
import pandas as pd

In [16]:
df = pd.DataFrame(list(zip(all_users, all_handles)), 
               columns =['screen_name', 'friend'])

In [17]:
df.head()

,screen_name,friend
0,ShippersUnbound,danharris80
1,ShippersUnbound,NatalieElphicke
2,ShippersUnbound,KermodeMovie
3,ShippersUnbound,JoshNorris
4,ShippersUnbound,timlevell


In [18]:
df.to_csv('../data/raw/'+keyword+'_journalist_friends.csv', index=False)

## 4. Quick inspection of data

In [12]:
from collections import Counter

In [13]:
counted = Counter(all_handles)

In [14]:
counted.most_common(50)

[('bbclaurak', 392),
 ('Peston', 368),
 ('afneil', 356),
 ('10DowningStreet', 353),
 ('bbcnickrobinson', 353),
 ('BBCBreaking', 334),
 ('paulwaugh', 333),
 ('faisalislam', 332),
 ('BBCPolitics', 331),
 ('ChrisMasonBBC', 330),
 ('BethRigby', 329),
 ('MichaelLCrick', 329),
 ('SamCoatesSky', 328),
 ('ShippersUnbound', 328),
 ('Keir_Starmer', 326),
 ('Ed_Miliband', 323),
 ('BBCNormanS', 321),
 ('tnewtondunn', 319),
 ('jonsnowC4', 317),
 ('PippaCrerar', 314),
 ('IsabelHardman', 312),
 ('PickardJE', 310),
 ('BBCNewsnight', 310),
 ('BorisJohnson', 308),
 ('tom_watson', 308),
 ('jimwaterson', 307),
 ('FraserNelson', 307),
 ('campbellclaret', 307),
 ('GuidoFawkes', 306),
 ('jeremycorbyn', 305),
 ('nicholaswatt', 301),
 ('BBCr4today', 298),
 ('David_Cameron', 294),
 ('alexwickham', 293),
 ('realDonaldTrump', 292),
 ('christopherhope', 292),
 ('suttonnick', 290),
 ('JPonpolitics', 290),
 ('maitlis', 287),
 ('Channel4News', 287),
 ('Kevin_Maguire', 286),
 ('hendopolis', 284),
 ('cathynewman', 284)